In [0]:
# Importing Relevant Libraries
import numpy as np
import math
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.feature import OneHotEncoder, StringIndexer, MinMaxScaler, VectorAssembler, StandardScaler
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.feature import *
from pyspark.sql.types import *
from pyspark.ml.linalg import *
from pyspark.sql.functions import *
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import PipelineModel
from pyspark.ml.stat import Correlation
import seaborn as sns
import pickle
import os
import matplotlib.pyplot as plt

In [0]:
# File location and type
file_location = "/FileStore/tables/train_features.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
features_data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
# File location and type
file_location = "/FileStore/tables/train_targets_scored-1.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
targets_data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:

# features_data = spark.read.load("/FileStore/tables/train_features.csv",
#                      format="csv", sep=":", inferSchema="true", header="true")

# targets_data = spark.read.load("/FileStore/tables/train_targets_scored-1.csv",
#                      format="csv", sep=":", inferSchema="true", header="true")

### An interesting pattern in the gene expressions and cell viability values

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/"))

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-datasets/COVID/,COVID/,0
dbfs:/databricks-datasets/README.md,README.md,976
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359
dbfs:/databricks-datasets/adult/,adult/,0
dbfs:/databricks-datasets/airlines/,airlines/,0
dbfs:/databricks-datasets/amazon/,amazon/,0
dbfs:/databricks-datasets/asa/,asa/,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0


In [0]:
#dbutils.fs.rm("dbfs:/FileStore/tables/ratings.csv")

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/"))

path,name,size
dbfs:/FileStore/import-stage/,import-stage/,0
dbfs:/FileStore/plots/,plots/,0
dbfs:/FileStore/shared_uploads/,shared_uploads/,0
dbfs:/FileStore/tables/,tables/,0


In [0]:
features_df = features_data.toPandas()

In [0]:
# Checking progression of gene_expressions values

# gene_expression values for 1st sample
features_df.iloc[0, 4:4+772].plot()
plt.title('Gene Expressions')
plt.show()

# Sorted gene_feature values for 1st sample
features_df.iloc[0, 4:4+772].sort_values().plot()
plt.title('Gene Expressions (Sorted by Values)')
plt.show()

java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17697); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17697)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:62)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:106)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:443)
	at com.databricks.backend.daemon.driver.FileStorePlotImageSaver.savePlotImage(FileStorePlotImageSaver.scala:33)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.generateImageResult(PythonDriverLocal.scala:1153)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.computeListResultsItem(PythonDriverLocal.scala:956)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$getResultBufferInternal$1(PythonDriverLocal.scala:1037)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:981)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:538)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.outputSuccess(PythonDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$repl$8(PythonDriverLocal.scala:382)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:369)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.data

In [0]:
# Checking progression of cell viability values

# cell viability values for 1st sample
features_df.iloc[0, 4+772:].plot()
plt.title('Cel Viability')
plt.show()

# Sorted gene_feature values for 1st sample
features_df.iloc[0, 4+772:].sort_values().plot()
plt.title('Cell Viability (Sorted by Values)')
plt.show()

java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17211); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17211)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:62)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:106)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:443)
	at com.databricks.backend.daemon.driver.FileStorePlotImageSaver.savePlotImage(FileStorePlotImageSaver.scala:33)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.generateImageResult(PythonDriverLocal.scala:1153)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.computeListResultsItem(PythonDriverLocal.scala:956)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$getResultBufferInternal$1(PythonDriverLocal.scala:1037)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:981)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:538)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.outputSuccess(PythonDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$repl$8(PythonDriverLocal.scala:382)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:369)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.data

As can be seen from the above graphs, gene expression and cell viability data (when sorted by values) might be interpreted as some time series data (same pattern exists for all samples). This info is further used in feature selection part, wherein max and min values of gene expression and cell viability are added as features for every sample

### Pre-Processing

In [0]:
# Dropping samples with cp_type = 'ctl_vehicle' (placebo group would have no activations in any MoAs) 
features_and_targets = features_data.join(targets_data, features_data.sig_id == targets_data.sig_id, how='inner').drop(targets_data.sig_id)
features_and_targets = features_and_targets.filter("cp_type = 'trt_cp'")

In [0]:
# One hot encoding cp_dose

# Indexing different categories in cp_type and storing in column 'cp_type_index'
string_indexer = StringIndexer(inputCol = 'cp_dose', outputCol = 'cp_dose_index')
features_and_targets = string_indexer.fit(features_and_targets).transform(features_and_targets).drop('cp_dose')

# One hot encoded and stored in cp_type_vector column
encoder = OneHotEncoder(inputCols=['cp_dose_index'],
                        outputCols=['cp_dose'], dropLast = False)

features_and_targets = encoder.fit(features_and_targets).transform(features_and_targets).drop('cp_dose_index')

In [0]:
# Label Encoding cp_time
assembler = VectorAssembler(inputCols = ['cp_time'], outputCol = 'cp_time_vector')
features_and_targets = assembler.transform(features_and_targets).drop('cp_time')

mm_scaler = MinMaxScaler(inputCol = 'cp_time_vector', outputCol = 'cp_time_scaled_vector', min = 0, max = 2)
features_and_targets = mm_scaler.fit(features_and_targets).transform(features_and_targets).drop('cp_time_vector')

firstElement=udf(lambda v:float(v[0]),FloatType())
features_and_targets = features_and_targets.withColumn('cp_time', firstElement(features_and_targets['cp_time_scaled_vector'])).drop('cp_time_scaled_vector')

### Stratified train-test split

In [0]:
'''
As our dataset is imbalanced, stratified train-test split was done so that test score is relevant. To perform stratified
split, I encoded all possible columns of target labels and then stratified was split was done based on the encoded value.
This ensures that, for every combination of target labels in the dataset, there are some samples both in train and test set
'''

# Encoding target labels' combination
def vector_to_string(a):
  s=''
  for i in a:
    s = s + str(i) + ' '
  s.strip()
  return s
    
  
vector_to_string = udf(vector_to_string)   
spark.udf.register("vector_to_string", vector_to_string)

# Target columns names
target_names = []
for feature in targets_data.columns:
  if feature != 'sig_id':
    target_names.append(feature)
    
temp_df = features_and_targets.withColumn('target_vector', (vector_to_string(array([features_and_targets[col] for col in target_names])))).select(['sig_id', 'target_vector'])
string_indexer = StringIndexer(inputCol = 'target_vector', outputCol = 'target')
string_indexer_model = string_indexer.fit(temp_df)
temp_df = string_indexer_model.transform(temp_df).drop('target_vector')

data = features_and_targets.join(temp_df, features_and_targets.sig_id == temp_df.sig_id, how = 'inner').drop(temp_df.sig_id)

unique_targets = data.select('target').distinct().collect()

# Train-test split (80-20 split)
fraction = {}
split_ratio = 0.2
for row in unique_targets:
  fraction[row.target] = split_ratio
  
test_data = data.sampleBy('target', fractions=fraction, seed = 0)
data.createOrReplaceTempView('data_table')
test_data.createOrReplaceTempView('test_data_table')
train_data = spark.sql('select * from data_table as t1 where t1.sig_id not in (select t2.sig_id from test_data_table as t2)') 

# Asserting no data leakage
train_data.createOrReplaceTempView('train_data_table')
count = spark.sql('select * from train_data_table as t1 where t1.sig_id in (select t2.sig_id from test_data_table as t2)').count()
assert count == 0

train_data = train_data.drop('target')
test_data = test_data.drop('target')

In [0]:
gene_feature_names = [] # list containing gene expression names
cell_feature_names = [] # list containing cell viability names

for i in range(772):
  gene_feature_names.append('g-' + str(i))

for i in range(100):
  cell_feature_names.append('c-' + str(i))

other_feature_names = ['sig_id', 'cp_type', 'cp_time', 'cp_dose']

target_names = [col for col in train_data.columns if col not in gene_feature_names + cell_feature_names + other_feature_names]

### Feature Engineering

In [0]:
train_data.head(5)

Out[20]: [Row(cp_type='trt_cp', g-0=-0.4857, g-1=0.5012, g-2=0.146, g-3=-0.1922, g-4=0.1351, g-5=0.195, g-6=0.2093, g-7=-0.068, g-8=-0.7707, g-9=-0.263, g-10=0.0707, g-11=0.2798, g-12=4.288, g-13=-0.6665, g-14=-0.107, g-15=0.1977, g-16=0.2942, g-17=-1.15, g-18=-0.0523, g-19=-0.0084, g-20=-0.6341, g-21=-0.0228, g-22=-0.8183, g-23=0.3155, g-24=0.0008, g-25=0.9218, g-26=0.2328, g-27=-0.4865, g-28=0.0576, g-29=0.497, g-30=-0.5037, g-31=0.272, g-32=1.273, g-33=-0.1894, g-34=0.5895, g-35=-0.7043, g-36=-0.8822, g-37=0.1045, g-38=-0.4023, g-39=-0.0219, g-40=-0.5882, g-41=-0.4678, g-42=0.9218, g-43=0.6414, g-44=-0.1509, g-45=0.4782, g-46=0.2928, g-47=-0.8182, g-48=-0.6547, g-49=-0.1776, g-50=-0.2371, g-51=0.0579, g-52=1.07, g-53=-0.4622, g-54=0.3732, g-55=0.6481, g-56=0.4356, g-57=0.3928, g-58=-0.4378, g-59=0.476, g-60=0.1549, g-61=0.0393, g-62=0.8866, g-63=0.5713, g-64=0.7702, g-65=-0.697, g-66=-0.0374, g-67=0.5941, g-68=0.5811, g-69=-0.4297, g-70=0.5809, g-71=0.2733, g-72=0.7232, g-73=-0.9363, g-74=-0.4319, g-75=0.4925, g-76=0.3758, g-77=0.0027, g-78=0.2919, g-79=0.2612, g-80=0.0228, g-81=-0.0077, g-82=1.846, g-83=-0.0767, g-84=3.748, g-85=-0.469, g-86=0.0644, g-87=0.3959, g-88=1.281, g-89=0.3769, g-90=-0.6562, g-91=-1.504, g-92=-0.1529, g-93=0.0691, g-94=-0.5903, g-95=-1.512, g-96=-0.1572, g-97=-0.478, g-98=0.2583, g-99=-0.2069, g-100=-0.2292, g-101=0.4357, g-102=-0.831, g-103=-0.9604, g-104=-0.5892, g-105=0.6649, g-106=-0.3655, g-107=-0.4348, g-108=-0.6077, g-109=0.0394, g-110=-0.5788, g-111=-0.5994, g-112=0.2496, g-113=-2.207, g-114=-0.1525, g-115=-0.5971, g-116=-0.1478, g-117=0.2501, g-118=0.3553, g-119=0.4242, g-120=0.2712, g-121=-0.4805, g-122=0.1168, g-123=-0.0788, g-124=-0.3678, g-125=0.9211, g-126=-0.5377, g-127=1.055, g-128=0.3363, g-129=-0.5031, g-130=1.302, g-131=-0.4801, g-132=0.2607, g-133=-0.218, g-134=-0.0587, g-135=-0.3885, g-136=0.5439, g-137=-0.4789, g-138=0.8056, g-139=0.4945, g-140=-0.7505, g-141=-0.2702, g-142=-0.2593, g-143=-1.256, g-144=-0.4229, g-145=0.8642, g-146=-1.126, g-147=-0.2477, g-148=-0.0154, g-149=-0.4886, g-150=0.0973, g-151=-0.2057, g-152=0.3119, g-153=0.5791, g-154=-0.5667, g-155=-0.9732, g-156=-0.1101, g-157=0.3698, g-158=-0.0571, g-159=1.931, g-160=-0.4399, g-161=-2.925, g-162=-0.0917, g-163=-0.8879, g-164=0.6716, g-165=0.1585, g-166=-0.2544, g-167=-0.0524, g-168=-0.012, g-169=-0.9724, g-170=-0.9239, g-171=-0.379, g-172=-0.378, g-173=0.1173, g-174=1.395, g-175=0.1526, g-176=-0.077, g-177=-0.7293, g-178=0.7594, g-179=-0.2486, g-180=0.0501, g-181=-0.263, g-182=0.542, g-183=0.989, g-184=-0.4324, g-185=-0.0877, g-186=-0.3465, g-187=-0.3366, g-188=-0.2749, g-189=-0.2592, g-190=0.2634, g-191=-0.7479, g-192=-1.239, g-193=-0.139, g-194=-0.3124, g-195=0.5488, g-196=-2.164, g-197=0.1267, g-198=0.4287, g-199=-0.4151, g-200=0.6421, g-201=-0.3402, g-202=-1.188, g-203=1.075, g-204=0.6671, g-205=-0.1618, g-206=-0.028, g-207=0.5291, g-208=-0.277, g-209=0.3092, g-210=0.2303, g-211=-0.1982, g-212=-0.172, g-213=-0.1787, g-214=1.077, g-215=-0.4636, g-216=-0.1188, g-217=-0.613, g-218=0.0102, g-219=-0.0583, g-220=0.0076, g-221=-0.2048, g-222=-1.044, g-223=0.741, g-224=1.604, g-225=0.7385, g-226=-0.4726, g-227=0.274, g-228=-0.3125, g-229=-0.4165, g-230=-0.499, g-231=-0.1294, g-232=0.3205, g-233=-0.9682, g-234=0.2826, g-235=-0.6592, g-236=0.2648, g-237=-0.6651, g-238=0.399, g-239=0.1072, g-240=-0.8798, g-241=-0.5771, g-242=-1.308, g-243=-0.0571, g-244=-0.3413, g-245=0.3261, g-246=0.5958, g-247=0.7042, g-248=0.0454, g-249=-0.8743, g-250=0.2055, g-251=0.1034, g-252=0.0409, g-253=-0.362, g-254=-0.0046, g-255=-0.8018, g-256=-0.1639, g-257=0.645, g-258=-0.0221, g-259=0.2084, g-260=-0.2331, g-261=-0.2209, g-262=0.3999, g-263=0.2615, g-264=-0.0929, g-265=5.13, g-266=0.2743, g-267=0.3633, g-268=-0.6918, g-269=-0.7534, g-270=0.0568, g-271=-0.2136, g-272=0.2958, g-273=-0.5807, g-274=-0.2529, g-275=-0.213, g-276=0.4087, g-277=-0.0498, g-278=-1.312, g-279=0.1222, g-280=-0.984, g-281=-2.087, g-282=-0.4509, g-283=-0.074, g-284=-1.0

In [0]:
# Engineering new features: max and min values for gene expression and cell viability data


max_gene_train_df = train_data.withColumn("gene_max", F.greatest(*gene_feature_names)).select('sig_id', 'gene_max')
min_gene_train_df = train_data.withColumn("gene_min", F.least(*gene_feature_names)).select('sig_id', 'gene_min')
max_cell_train_df = train_data.withColumn("cell_max", F.greatest(*cell_feature_names)).select('sig_id', 'cell_max')
min_cell_train_df = train_data.withColumn("cell_min", F.least(*cell_feature_names)).select('sig_id', 'cell_min')

mean_gene_train_df = train_data.withColumn("gene_ma", F.max(gene_feature_names)).select('sig_id', 'gene_ma')
mean_cell_train_df = train_data.withColumn("cell_max", F.mean(*cell_feature_names)).select('sig_id', 'cell_max')







max_gene_test_df = test_data.withColumn("gene_max", F.greatest(*gene_feature_names)).select('sig_id', 'gene_max')
min_gene_test_df = test_data.withColumn("gene_min", F.least(*gene_feature_names)).select('sig_id', 'gene_min')
max_cell_test_df = test_data.withColumn("cell_max", F.greatest(*cell_feature_names)).select('sig_id', 'cell_max')
min_cell_test_df = test_data.withColumn("cell_min", F.least(*cell_feature_names)).select('sig_id', 'cell_min')

### Feature Selection

In [0]:
# Finding Correlation among cont. gene expression and cell viability data to remove all redundant features

# Correlation among gene expressions and cell viability

cont_features = gene_feature_names + cell_feature_names
cont_features_df = train_data.select(cont_features)

va = VectorAssembler(inputCols = cont_features, outputCol = 'features')
cont_features_df = va.transform(cont_features_df)

corr_df = Correlation.corr(cont_features_df, 'features', 'pearson').collect()[0][0]

corr_mat = corr_df.toArray()


In [0]:
plt.figure(figsize=(40,20))
sns.heatmap(corr_mat, xticklabels = gene_feature_names + cell_feature_names, yticklabels=gene_feature_names + cell_feature_names)
plt.xticks(rotation = 90, fontsize = 5)
plt.show()

java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17388); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17388)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:62)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:106)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:443)
	at com.databricks.backend.daemon.driver.FileStorePlotImageSaver.savePlotImage(FileStorePlotImageSaver.scala:33)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.generateImageResult(PythonDriverLocal.scala:1153)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.computeListResultsItem(PythonDriverLocal.scala:956)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$getResultBufferInternal$1(PythonDriverLocal.scala:1037)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:981)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:538)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.outputSuccess(PythonDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$repl$8(PythonDriverLocal.scala:382)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:369)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.data

In [0]:
# Feature Selection: Removing highly correlated continuos features
threshold = 0.7 
columns = np.full((corr_mat.shape[0], ), True, dtype = bool)
for i in range(corr_mat.shape[0]):
  for j in range(i+1, corr_mat.shape[0]):
    if not columns[j]:
      continue
    if corr_mat[i,j] >= threshold or corr_mat[i,j] <= -threshold :
      if columns[j]:
        columns[j] = False

selected_features = [(gene_feature_names + cell_feature_names)[i] for i, j in enumerate(columns) if j==True]

In [0]:
len(selected_features) # Number of features left after removing correlated  features

Out[27]: 530

### Standardizing selected features

Adding engineered  features and then standardizing all continuous features

In [0]:
train_data = train_data.select(other_feature_names + selected_features + target_names).join(max_gene_train_df, on = 'sig_id', how = 'inner').join(min_gene_train_df, on = 'sig_id', how = 'inner').join(max_cell_train_df, on = 'sig_id', how = 'inner').join(min_cell_train_df, on = 'sig_id', how = 'inner')
test_data = test_data.select(other_feature_names + selected_features + target_names).join(max_gene_test_df, on = 'sig_id', how = 'inner').join(min_gene_test_df, on = 'sig_id', how = 'inner').join(max_cell_test_df, on = 'sig_id', how = 'inner').join(min_cell_test_df, on = 'sig_id', how = 'inner')

In [0]:
selected_features = selected_features + ['gene_max', 'gene_min', 'cell_max', 'cell_min']

In [0]:
va = VectorAssembler(inputCols = selected_features, outputCol = 'feature_vector')

train_data = va.transform(train_data)
test_data = va.transform(test_data)

ss = StandardScaler(withMean=True, inputCol = 'feature_vector', outputCol='features')
ss_model = ss.fit(train_data)

train_data = ss_model.transform(train_data).drop('feature_vector')
test_data = ss_model.transform(test_data).drop('feature_vector')

In [0]:

max_gene_train_df = train_data.withColumn("gene_max", F.greatest(*gene_feature_names)).select('sig_id', 'gene_max')
min_gene_train_df = train_data.withColumn("gene_min", F.least(*gene_feature_names)).select('sig_id', 'gene_min')
max_cell_train_df = train_data.withColumn("cell_max", F.greatest(*cell_feature_names)).select('sig_id', 'cell_max')
min_cell_train_df = train_data.withColumn("cell_min", F.least(*cell_feature_names)).select('sig_id', 'cell_min')
mean_gene_train_df = train_data.withColumn("gene_ma", F.mean(gene_feature_names)).select('sig_id', 'gene_ma')
mean_cell_train_df = train_data.withColumn("cell_max", F.mean(*cell_feature_names)).select('sig_id', 'cell_max')







max_gene_test_df = test_data.withColumn("gene_max", F.greatest(*gene_feature_names)).select('sig_id', 'gene_max')
min_gene_test_df = test_data.withColumn("gene_min", F.least(*gene_feature_names)).select('sig_id', 'gene_min')
max_cell_test_df = test_data.withColumn("cell_max", F.greatest(*cell_feature_names)).select('sig_id', 'cell_max')
min_cell_test_df = test_data.withColumn("cell_min", F.least(*cell_feature_names)).select('sig_id', 'cell_min')
mean_gene_train_df = train_data.withColumn("gene_ma", F.mean(gene_feature_names)).select('sig_id', 'gene_ma')
mean_cell_train_df = train_data.withColumn("cell_max", F.mean(*cell_feature_names)).select('sig_id', 'cell_max')




### Dimensionality Reduction (PCA)

In [0]:
comp = 300 
pca = PCA(k = comp, inputCol='features', outputCol='pca_features')
pca_model = pca.fit(train_data)
cum_var = np.cumsum(pca_model.explainedVariance.toArray())

In [0]:
cum_var = np.cumsum(pca_model.explainedVariance.toArray())

In [0]:
cum_var

Out[37]: array([0.12123983, 0.16657677, 0.20675103, 0.24233472, 0.26986472,
 0.29336871, 0.31423415, 0.33362091, 0.34978106, 0.36379207,
 0.3772279 , 0.38873756, 0.39904703, 0.40831919, 0.41723675,
 0.4253879 , 0.43329505, 0.44051004, 0.44697082, 0.45306984,
 0.45875591, 0.46422391, 0.46927757, 0.47421213, 0.47902359,
 0.48359915, 0.4879248 , 0.49211604, 0.49617011, 0.50018887,
 0.50406569, 0.50791728, 0.51160395, 0.51528416, 0.51876822,
 0.52220602, 0.5254999 , 0.52871538, 0.53186868, 0.53492708,
 0.53795955, 0.54093606, 0.54382536, 0.54662808, 0.54938044,
 0.55210596, 0.55479104, 0.55742103, 0.56002859, 0.56258354,
 0.56511896, 0.56760708, 0.57006432, 0.57251326, 0.57494908,
 0.57734755, 0.57973844, 0.58210413, 0.58443865, 0.58674572,
 0.58901843, 0.59127735, 0.59349754, 0.59569964, 0.59789757,
 0.60007774, 0.60224241, 0.60440171, 0.60653849, 0.60866232,
 0.61077212, 0.61287693, 0.61496253, 0.61702942, 0.61907951,
 0.62112414, 0.62315714, 0.6251765 , 0.62716539, 0.62914409,
 0.63111396, 0.63308055, 0.63503119, 0.63697014, 0.63888313,
 0.64078639, 0.64268411, 0.64457298, 0.64645328, 0.64832423,
 0.65017496, 0.65202499, 0.65384904, 0.65566573, 0.65747253,
 0.65927449, 0.66107477, 0.66285759, 0.66462501, 0.66638915,
 0.66813029, 0.66986579, 0.67159065, 0.67330534, 0.67501186,
 0.67671308, 0.67839406, 0.68006303, 0.681718 , 0.68336816,
 0.68501218, 0.68663763, 0.68825343, 0.68985905, 0.69146114,
 0.6930507 , 0.69463288, 0.69621068, 0.69777129, 0.6993181 ,
 0.70085631, 0.70238646, 0.70390999, 0.70542687, 0.70693525,
 0.70843185, 0.70991832, 0.7113944 , 0.71286221, 0.71432708,
 0.71578226, 0.71723433, 0.71867922, 0.72011952, 0.7215464 ,
 0.72296885, 0.72438876, 0.72578774, 0.72718391, 0.7285719 ,
 0.72994755, 0.73131179, 0.73267157, 0.73402787, 0.73537919,
 0.73672457, 0.73806572, 0.73940098, 0.74073186, 0.74205304,
 0.74337321, 0.74468185, 0.74598678, 0.74728843, 0.74857549,
 0.7498549 , 0.75112737, 0.75239002, 0.75364839, 0.75490596,
 0.75614796, 0.75738604, 0.75861843, 0.75984753, 0.76107165,
 0.76229046, 0.76350514, 0.76471378, 0.76591461, 0.76711003,
 0.76830227, 0.76949377, 0.7706732 , 0.77184709, 0.77301269,
 0.7741738 , 0.77532992, 0.77647987, 0.77762802, 0.7787744 ,
 0.77991664, 0.78105459, 0.78218769, 0.78331505, 0.78443246,
 0.78554418, 0.78665354, 0.78775829, 0.78885898, 0.78995523,
 0.79104986, 0.79213933, 0.79322372, 0.79430348, 0.79538029,
 0.79645306, 0.79751969, 0.79857978, 0.79963766, 0.80069249,
 0.8017416 , 0.80278857, 0.80382928, 0.80486662, 0.80589811,
 0.80692669, 0.8079543 , 0.80897613, 0.80999457, 0.81101128,
 0.8120243 , 0.81302972, 0.81403181, 0.81503227, 0.81602807,
 0.81702017, 0.81800493, 0.81898654, 0.81996736, 0.82094118,
 0.82190998, 0.82287738, 0.82384183, 0.82480011, 0.82575653,
 0.82670894, 0.82765309, 0.82859377, 0.82953368, 0.83046896,
 0.8313998 , 0.83232444, 0.83324731, 0.83416762, 0.83508528,
 0.83600057, 0.83691236, 0.83781884, 0.83872175, 0.83962251,
 0.84051779, 0.84141237, 0.84230155, 0.84318862, 0.84407264,
 0.84495524, 0.84583187, 0.84670635, 0.84757897, 0.84844769,
 0.84931067, 0.85017314, 0.851034 , 0.85188931, 0.85274312,
 0.85359395, 0.85444244, 0.85528421, 0.85612526, 0.85696249,
 0.85779631, 0.85862765, 0.85945498, 0.8602808 , 0.8611029 ,
 0.86192025, 0.86273544, 0.86354898, 0.86435981, 0.86516496,
 0.86596528, 0.86676378, 0.86755933, 0.86835204, 0.86914045,
 0.86992529, 0.87070962, 0.87149332, 0.87227026, 0.87304391,
 0.87381632, 0.87458797, 0.87535721, 0.87612303, 0.8768865 ,
 0.87764706, 0.8784064 , 0.87916373, 0.87991945, 0.88067054,
 0.88142116, 0.88216651, 0.88291156, 0.88365461, 0.8843931 ,
 0.88512841, 0.88585944, 0.88658809, 0.88731293, 0.88803633])

In [0]:
xlabels = ['pc-' + str(i+1) for i in range(k)]
plt.figure(figsize=(20,10))
plt.plot(xlabels, cum_var*100)
plt.xticks(rotation = 90)
plt.xlabel('Principal Components')
plt.ylabel('Cumulative Sum Variance')
plt.title('Cumulative Sum Variance vs Principal Components')
plt.show()

java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17679); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17679)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:62)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:106)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:443)
	at com.databricks.backend.daemon.driver.FileStorePlotImageSaver.savePlotImage(FileStorePlotImageSaver.scala:33)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.generateImageResult(PythonDriverLocal.scala:1153)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.computeListResultsItem(PythonDriverLocal.scala:956)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$getResultBufferInternal$1(PythonDriverLocal.scala:1037)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:981)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:538)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.outputSuccess(PythonDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$repl$8(PythonDriverLocal.scala:382)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:899)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:369)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.data

In [0]:
train_data = pca_model.transform(train_data).drop('features')
test_data = pca_model.transform(test_data).drop('features')

In [0]:
train_data = train_data.select(['cp_time', 'cp_dose', 'pca_features'] + target_names)
test_data = test_data.select(['cp_time', 'cp_dose', 'pca_features'] + target_names)

### Traininig and Testing

In [0]:
def logloss(predictedProb, trueLabel):
    loss = (-trueLabel*np.log(predictedProb + 10e-15) - (1-trueLabel)*(np.log(1-predictedProb))).mean()
    return loss

In [0]:
loss = 0
losses = [] # List to store log loss error for individual target labels
untrained_labels = [] # List to store target labels for which model was not trained

for i, label in enumerate(target_names):
  
  X_train = train_data.select(['cp_time', 'cp_dose', 'pca_features'] + [label])
  X_test = test_data.select(['cp_time', 'cp_dose', 'pca_features'] + [label])
  one_count_train = X_train.filter(X_train[label] == 1).count()
  zero_count_train = X_train.filter(X_train[label] == 0).count()
  
#   If there are no activations for the label in the training dataset, no need to train the model (prediction would be all 0's)  
  if one_count_train == 0:
    untrained_labels.append(label)
    n = X_test.count()
    prob = np.array([0]*n)
    label = np.array(X_test.select(label).toPandas())[:,0]
    l = logloss(prob, label)
    print('No need to train the model. Will always output 0 for this label')
    
# Handling Data Imbalance: if number of samples with label 0 is more than twice the samples with label 1 - undersample
# majority class samples by half and oversample minority class samples so that counts are same 
  else:  
    factor = zero_count_train/one_count_train
    if factor>=2:
      zero_samples = X_train.filter(X_train[label]==0).sample(fraction = 0.5, withReplacement = False,  seed = 0) 
      one_samples = spark.createDataFrame(X_train.filter(X_train[label]==1).toPandas().sample(n = int(X_train.filter(X_train[label]==0).count()/2), replace = True,  random_state = 0))
      X_train = zero_samples.union(one_samples).orderBy(rand())
    
    va = VectorAssembler(inputCols = ['cp_time', 'cp_dose', 'pca_features'], outputCol = 'features')
    X_train = va.transform(X_train)
    X_test = va.transform(X_test)

    gbt = GBTClassifier(labelCol = label)
    print('Training GBT')
    gbtModel = gbt.fit(X_train)
    print('GBT Trained')
    pred_df = gbtModel.transform(X_test)
    evaluator = MulticlassClassificationEvaluator(labelCol=label, metricName='logLoss')
    l= evaluator.evaluate(pred_df)
    
  losses.append(l)
  print(f'Label Loss for {i}: {label} = {l}')
  loss+=l
  print(f'Hamming Loss after {i}: {label} = {loss/(i+1)}')
  d = {'losses': losses, 'untrained_labels': untrained_labels }
  f = open('data1.pkl', 'wb')
  pickle.dump(d, f)
  f.close()  
  print('################################')
  X_train.unpersist()
  X_test.unpersist()
  pred_df.unpersist()  

/databricks/spark/python/pyspark/sql/pandas/conversion.py:93: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field cp_dose. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Context: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Could not convert (2,[0],[1.0]) with type SparseVector: did not recognize Python value type when inferring an Arrow data type
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Training GBT
GBT Trained

In [0]:
with open('data1.pkl', 'rb') as f:
    data = pickle.load(f)

In [0]:
len(data['losses'])

In [0]:
data['untrained_labels']

In [0]:
plt.figure(figsize = (60,10))
plt.plot(target_names, data['losses'])
plt.title('Logg Loss per target label', fontsize = 30)
plt.xlabel('target label', fontsize=30)
plt.ylabel('Avg BCE loss', fontsize = 30)
plt.xticks(rotation = 90, fontsize = 14)
plt.show()

In [0]:
def sum1(l):
    s = 0
    for i in l:
        s+=i
    return s

In [0]:
hamming_loss = [sum1(data['losses'][:i+1])/(i+1) for i in range(len(data['losses']))]

In [0]:
plt.figure(figsize = (60,10))
plt.plot(target_names, hamming_loss)
plt.title('Progression of hamming loss', fontsize = 30)
plt.xlabel('target label', fontsize=30)
plt.ylabel('Hamming loss', fontsize = 30)
plt.xticks(rotation = 90, fontsize = 14)
plt.yticks(fontsize = 30)
plt.show()

In [0]:
# Storing some info about training dynamics

data1 = data

In [0]:
data1['train_data'] = train_data.toPandas()
data1['test_data'] = test_data.toPandas()
data1['hamming_loss'] = hamming_loss
data1['variance'] = pca_model.explainedVariance.toArray()
data1['cumulative_variance'] = cum_var
data1['correlation_matrix'] = corr_mat
data1['selected_features'] = selected_features

In [0]:
with open('data.pkl', 'wb') as f:
    pickle.dump(data1, f)
